In [30]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from datetime import date
from ICBAnalytics import GetData

pd.set_option('display.max_columns', None)

def actualizacion_var():

    try:
        query = '''
        SELECT *
        FROM inventarios
        '''
        data = GetData().get_data_postgresql(query)
        print("{} Total records in the query!".format(data.shape[0]))

    except Exception as e: 
            print('Failed to call the query.', e)

                
    meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
    data['AÑO'] = data['AÑO'].apply(lambda x: int(x))
    data['MES'] = data['MES'].apply(lambda x: meses.index(x.upper())+1 if x.upper() in meses else None)
    data = data.sort_values(by = 'fecha', ascending = True)
    data['Mes completo'] = (data['AÑO'] - 2020)*12 + data['MES']

    data['Tipo inventario anterior'] = data.groupby(['Ruta'])['Tipo Inventario'].shift()
    data['Valor total anterior'] = data.groupby(['Ruta'])['Valor total'].shift()
    data['Inventario anterior'] = data.groupby(['Ruta'])['Inventario'].shift()
    data['Mermas anterior'] = data.groupby(['Ruta'])['Mermas'].shift()
    data['Vencidos anterior'] = data.groupby(['Ruta'])['Vencidos'].shift()
    data['Siniestro anterior'] = data.groupby(['Ruta'])['Siniestro'].shift()

    data['fecha'] = pd.to_datetime(data['fecha'])
    data["fecha_anterior"] = data.groupby(['Ruta'])['fecha'].shift()
    dia_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
    data['Dia semana'] = data['fecha'].apply(lambda x: dia_semana[x.weekday()])

    data['Siniestro'].fillna(0, inplace = True)
    data['Siniestro anterior'].fillna(0, inplace = True)

        # Cuenta de inventarios acumulador por vendedor y administrativo
    data['Inventarios vendedor'] = data.groupby(['Vendedor']).cumcount()
    data['Inventarios administrativo'] = data.groupby(['Administrativo']).cumcount()

    cols = ['Ruta', 'Rut Gestor Responsable', 'Vendedor', 'fecha', 'Inventario', 'Mermas', 'Vencidos',
            'Siniestro', 'Valor total', 'Inventario anterior', 'Mermas anterior', 'Vencidos anterior',
            'Siniestro anterior', 'Valor total anterior', 'MES', 'AÑO', 'inventariado', 'Administrativo', 'Tipo Inventario', 'Orden Camion', 'Tipo inventario anterior']
    df = data[cols]

    df.sort_values(by = 'fecha', ascending = True, inplace = True)
    df.reset_index(drop = True, inplace = True)

    df.drop(['Orden Camion'], axis = 1, inplace = True)
    df['Ruta con diferencia'] = (df['Valor total'] <= -40000)*1
    df['Ruta con diferencia +1'] = df.groupby(['Rut Gestor Responsable'])['Ruta con diferencia'].shift(-1)

    #df['Ruta'] = df.groupby(['Rut Gestor Responsable'])['Ruta'].shift(-1)
    #df['Rut Gestor Responsable'] = df['Rut Gestor Responsable'].str.upper()
    #df['Vendedor'] = df.groupby(['Ruta'])['Vendedor'].shift(-1)

    df['Semana'] = pd.to_datetime(df['fecha']).apply(lambda x: x.week)

    query = '''

    SELECT MAX(fecha)
    FROM inv_ampliado
        
    '''
    fecha_max = GetData().get_data_postgresql(query)
    fecha_max = pd.to_datetime(fecha_max['max'][0])
    year = fecha_max.year
    month = fecha_max.month
    today = date.today()
        
    print('Max Year: {}\nMax month: {}'.format(year, month))

    query = """
    WITH deleted AS (DELETE FROM inv_ampliado 
                        WHERE CAST(to_char("fecha", 'YYYY') AS INTEGER) = {year}
                        AND CAST(to_char("fecha", 'MM') AS INTEGER) >= {month} RETURNING *) SELECT count(*) FROM deleted;

        """.format(year = year, month = month)

    rows_deleted = GetData().get_data_postgresql(query)
    print('Rows deleted: {}'.format(rows_deleted))
    df = df.astype(object).where(pd.notnull(df), None)
        
    if type(fecha_max) != type(None):
        if today.year > year:
            df = df[((df['AÑO'] >= year) & (df['MES'] >= month)) | ((df['AÑO'] == today.year) & (df['MES'] == today.month))]
        else:
            df = df[(df['AÑO'] >= year) & (df['MES'] >= month)]

    print("{} Total records in the query!".format(df.shape[0]))

        
    query = '''
        
    INSERT INTO public.inv_ampliado(
        "Ruta",  "rut_gestor", "Vendedor", "fecha", "Inventario", "Mermas", "Vencidos", "Siniestro", "Valor total", "Inventario anterior", "Mermas anterior", "Vencidos anterior", "Siniestro anterior", "Valor total anterior", "MES", "AÑO", inventariado, "Administrativo", "Tipo Inventario", "Tipo Inventario anterior", "Ruta con diferencia", "Ruta con diferencia +1", "Semana")
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        '''

    cursor.fast_executemany = True
    cursor.executemany(query, df.values.tolist())
    cursor.commit()
    print("Query updated succesfully!")




    return df



In [32]:
df = actualizacion_var()
df.shape

Ok
PostgreSQL connection is closed
244 Total records in the query!


c:\Users\erucan\AppData\Local\miniconda3\envs\default\lib\site-packages\ICBAnalytics\get_data_api.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(self.query, cnxn)


(244, 29)

In [ ]:
df[df["Ruta"] == ]

In [ ]:
df[]